In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from PIL import Image
import matplotlib.pyplot as plt

# ---------------------------------------------------------
# 1️ CONFIGURATION ET DATAFRAME (Inchangé ou presque)
# ---------------------------------------------------------
dataset_path = r"C:\Users\rache\Downloads\archive\raw-img"
translate = {"cane": "dog", "cavallo": "horse", "elefante": "elephant", "farfalla": "butterfly", 
             "gallina": "chicken", "gatto": "cat", "mucca": "cow", "pecora": "sheep", 
             "scoiattolo": "squirrel", "ragno": "ragno"}

data = []
for folder in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder)
    if os.path.isdir(folder_path):
        label = translate.get(folder, folder)
        for file in os.listdir(folder_path):
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                data.append([os.path.join(folder_path, file), label])

df = pd.DataFrame(data, columns=["image_path", "label"])
num_classes = len(df['label'].unique())
label_mapping = dict(enumerate(df['label'].astype('category').cat.categories))

train_df, test_df = train_test_split(df, test_size=0.1, stratify=df['label'], random_state=42)

# ---------------------------------------------------------

# ---------------------------------------------------------
# 1.5️ CLASSES ET TRANSFORMS (Obligatoire pour que ça tourne)
# ---------------------------------------------------------
class AnimalDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        # On convertit les noms (dog, cat...) en chiffres (0, 1...)
        self.label_to_idx = {v: k for k, v in label_mapping.items()}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.iloc[idx, 0]
        image = Image.open(img_path).convert("RGB")
        label = self.label_to_idx[self.df.iloc[idx, 1]]
        if self.transform:
            image = self.transform(image)
        return image, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Initialisation de l'historique pour les graphiques
history = {'loss': [], 'accuracy': []}





# 2️ DATA LOADING (Ajusté : Batch Size = 32)


# ---------------------------------------------------------
BATCH_SIZE = 32 # Modifié selon ton tableau

train_loader = DataLoader(AnimalDataset(train_df, transform), batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(AnimalDataset(test_df, transform), batch_size=BATCH_SIZE)

# ---------------------------------------------------------

# 3️ MODÈLE 2 (Avec Dropout pour réduire l'overfitting)
# ---------------------------------------------------------
class AnimalCNN_V2(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2, 2),
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 28 * 28, 64),
            nn.ReLU(),
            # --- LA MODIFICATION EST ICI ---
            nn.Dropout(0.5), # On désactive 50% des connexions aléatoirement
            # -------------------------------
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

# N'oublie pas de recréer le modèle et l'optimizer

# 1. Définir le moteur (GPU ou CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. Créer l'objet du modèle ET l'envoyer sur la carte direct
model = AnimalCNN_V2(num_classes=num_classes).to(device)

# 3. Créer l'optimiseur (il va maintenant lier les paramètres qui sont déjà sur le GPU)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 4. Définir la fonction de perte
criterion = nn.CrossEntropyLoss()


# ---------------------------------------------------------
# 4️ ENTRAÎNEMENT (Paramètres Model 1)
# ---------------------------------------------------------
# Optimizer: Adam | Learning rate: default (0.001) | Momentum: No
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Epochs: 15 (comme spécifié)


for epoch in range(15):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        # Transfert vers la 5070
        images, labels = images.to(device), labels.to(device, dtype=torch.long)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    epoch_loss = running_loss/len(train_loader)
    epoch_acc = 100. * correct / total
    history['loss'].append(epoch_loss)
    history['accuracy'].append(epoch_acc)
    
    print(f"Epoch {epoch+1}/15 | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.2f}%")

# ---------------------------------------------------------

# 5️ VALIDATION LOOP (The "Real" Test)
# ---------------------------------------------------------
model.eval()  # Switch to evaluation mode (turns off Dropout)
val_correct = 0
val_total = 0

print("\n🔍 Running Validation on unseen images...")

with torch.no_grad(): # Disable gradient calculation (saves memory/time)
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        val_total += labels.size(0)
        val_correct += (predicted == labels).sum().item()

val_accuracy = 100 * val_correct / val_total




# 6 VISUALISATION (Remplace tes plots matplotlib)
# ---------------------------------------------------------
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.title('Loss')
plt.plot(history['loss'], color='blue')
plt.subplot(1, 2, 2)
plt.title('Accuracy')
plt.plot(history['accuracy'], color='green')
print(f"✅ Final Validation Accuracy: {val_accuracy:.2f}%")
plt.savefig('resultats_model_2.png')
plt.show()

print("ITS DONE ! 🎉")

In [ ]:
# ---------------------------------------------------------
# 7 VISUALISATION DES PRÉDICTIONS (Test sur 10 images)
# ---------------------------------------------------------
def visualize_predictions(model, test_loader, num_images=10):
    model.eval()
    images_shown = 0
    plt.figure(figsize=(15, 6)) # Taille de la fenêtre d'affichage

    # On récupère le nom des classes dans l'ordre
    class_names = [label_mapping[i] for i in range(len(label_mapping))]

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            for i in range(images.size(0)):
                if images_shown >= num_images:
                    break
                
                # Préparation de l'image pour l'affichage (on doit la remettre en format standard)
                img = images[i].cpu().numpy().transpose((1, 2, 0))
                mean = np.array([0.485, 0.456, 0.406])
                std = np.array([0.229, 0.224, 0.225])
                img = std * img + mean # Dénormalisation
                img = np.clip(img, 0, 1)

                ax = plt.subplot(2, 5, images_shown + 1)
                plt.imshow(img)
                
                # Couleur du texte : Vert si c'est bon, Rouge si c'est faux
                color = 'green' if preds[i] == labels[i] else 'red'
                
                plt.title(f"P: {class_names[preds[i].item()]}\n(V: {class_names[labels[i].item()]})", 
                          color=color, fontsize=10)
                plt.axis('off')
                images_shown += 1
            
            if images_shown >= num_images:
                break

    plt.tight_layout()
    plt.show()

# Appeler la fonction
visualize_predictions(model, test_loader, num_images=10)